In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import requests
from youtube_transcript_api import YouTubeTranscriptApi
import re
import time, datetime
import bisect
from langchain.llms import OpenAI
from typing import List

In [69]:
video_code = 'aVQQuoToTJA'

In [83]:
def chapters_and_time_and_images(video_code: str) -> List[str]:

    api_response_object: requests.Response = requests.get(f'https://yt.lemnoslife.com/videos?part=chapters&id={video_code}')
    api_response = api_response_object.json()

    chapters: List = []
    start_time: List= []
    images_link: List = []
    number_of_titles:int = len(api_response['items'][0]['chapters']['chapters'])
    
    for i in range(0,number_of_titles):
        chapters.append(api_response['items'][0]['chapters']['chapters'][i]['title'])
        start_time.append(api_response['items'][0]['chapters']['chapters'][i]['time'])
        images_link.append(api_response['items'][0]['chapters']['chapters'][i]['thumbnails'][1]['url'])

    
    # if 0 not in start_time:
    #     start_time.insert(0, 0)
    
    return chapters, start_time, images_link


In [86]:
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://dev-pypi.aganitha.ai
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.3 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [84]:
chapters_and_time_and_images(video_code)

(['Hallucinating Symmetric Protein Assemblies',
  'Incorporating these Short Linear Motives into Genova Design Protein Scaffolds',
  'Structural Addition',
  'Problem Statement',
  'Sampling Temperature',
  'Amino Acid Biases',
  'Sequence Recovery Maximum Accuracy',
  'Solution Conditions',
  'The Membrane Proteins',
  'Is Body Impedance Score Dependent on the Decoding Order',
  'What Applications Do You Envision Mpn Will Be Used for',
  'If a Background Has More Diversity at Lower Temperatures Could that Mean It Is More Designable'],
 [107, 336, 593, 716, 1548, 1668, 2424, 2667, 2917, 2938, 2965, 3075],
 ['https://i.ytimg.com/vi/aVQQuoToTJA/hqdefault_114433.jpg?sqp=-oaymwE9CNACELwBSFryq4qpAy8IARUAAAAAGAElAADIQj0AgKJDeAHwAQH4Af4JgALQBYoCDAgAEAEYZCBlKFQwDw==&rs=AOn4CLAWsuPZH7ETroeyS2k9HtBAn3XRpA',
  'https://i.ytimg.com/vi/aVQQuoToTJA/hqdefault_337133.jpg?sqp=-oaymwE9CNACELwBSFryq4qpAy8IARUAAAAAGAElAADIQj0AgKJDeAHwAQH4Af4JgALQBYoCDAgAEAEYZCBlKFQwDw==&rs=AOn4CLCah5fg9rV46D75l-2vsn35EZsu

In [72]:
def get_transcripts(video_code: str) -> List[str]:
    
    start: List = []
    text: List = []

    # Get the start time and text of closed captions from transcript API 
    trans = YouTubeTranscriptApi.get_transcript(video_code)
    for content in trans:
        start.append(content['start'])
        text.append(content['text'])

    return start, text

In [73]:
def find_lt(a, x):
    'Find rightmost value less than x'
    if x == '00:00:00' or x == 0:
        return 0
    i = bisect.bisect_left(a, x)
    if i:
        if a[i] == x:
            return i
        else:
            return i-1

In [92]:
chapters, start_time_api, images_url = chapters_and_time_and_images(video_code)
start, text = get_transcripts(video_code)

indexes: List = []
for j in start_time_api:
    indexes.append(find_lt(start, j))

In [75]:
indexes

[42, 134, 246, 296, 660, 717, 1043, 1165, 1276, 1284, 1298, 1342]

In [77]:
_ , modified_text = get_transcripts(video_code)

counter = -1
modified_text.insert(-1, "\n\n\n")
for i in indexes[::-1]:

    if i == 0:
        modified_text.insert(0, "\n\n")
        modified_text.insert(0, chapters[0])
    else:
        modified_text.insert(i-1, "\n\n")
        modified_text.insert(i-1, chapters[counter])
        modified_text.insert(i-1, "\n\n\n")
        counter -= 1


with open("tmp2.txt", "w+") as file:
     file.write(" ".join(modified_text))

In [ ]:
from PIL import Image

data = requests.get('https://i.ytimg.com/vi/aVQQuoToTJA/hqdefault_114433.jpg?sqp=-oaymwE9CNACELwBSFryq4qpAy8IARUAAAAAGAElAADIQj0AgKJDeAHwAQH4Af4JgALQBYoCDAgAEAEYZCBlKFQwDw==&rs=AOn4CLAWsuPZH7ETroeyS2k9HtBAn3XRpA').content
  
with open('img.jpg','wb') as f:
    f.write(data)
img = Image.open('img.jpg')
img.show()

In [96]:
llm = OpenAI(model_name = "gpt-3.5-turbo", temperature = 0)
idx = 0
prompt: str = """You are a youtube transcript punctuator. Given a youtube transcript, you should add all the punctuations wherever necessary without losing the context.
Below is the transcript from a youtube video.
Punctuate it and give it back in html format.

The first line is the title. Add two new lines after the heading and make the title bold and then punctuate it."""
counter = 0
for i in range(len(modified_text)):
    if modified_text[i] == "\n\n\n":
        chunked_text = modified_text[idx:i]
        chunk = " ".join(chunked_text)
        new_prompt = prompt + "\n\n" + chunk
        print(new_prompt)
        response: str = llm(new_prompt)
        print(response)
        with open(str(counter)+'.jpg','wb') as f:
            f.write(data)
        if 0 not in start_time_api:
            with open("test_chapters_2.html", "a+") as file:
                file.write(f"\n\n{response}")
                file.write(f"\n<a href=\"https://www.youtube.com/watch?v={video_code}&t={start[idx]}s\">{chunked_text[0]}</a>\n")
        else:
            data = requests.get(images_url[counter]).content
            with open("test_chapters_2.html", "a+") as file:
                file.write(f"\n\n{response}")
                file.write(f"\n<img src=\"{str(counter)+'.jpg'}\" alt=\"Not Applicable\" />")
                file.write(f"\n<a href=\"https://www.youtube.com/watch?v={video_code}&t={start[idx]}s\">{chunked_text[0]}</a>\n")
                counter += 1
        idx = i+1
        

You are a youtube transcript punctuator. Given a youtube transcript, you should add all the punctuations wherever necessary without losing the context.
Below is the transcript from a youtube video.
Punctuate it and give it back in html format.

The first line is the title. Add two new lines after the heading and make the title bold and then punctuate it.

hi everyone it's my pleasure and today to introduce Eustis Eustis is a postdoc at the baker Lab at the University of Washington he obtained his bachelor's and Master's in mathematics and his PhD in biophysics all from the University of Cambridge today Eustis will talk about his work on protein mpnn I'm sure all of us have seen the work recently come out and in science and we're all very excited to uh to hear what you've got for us so I think we're going to keep questions uh for the end but feel free to to put them in the chat in the meantime take it away you sis uh thank you for the introduction um so I'll be presenting on party MPN a

KeyboardInterrupt: 